In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference')
print(os.getcwd())

/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference


In [2]:
import pickle
import pprint

import torch

from utils import general_utils, datasets
from utils.datasets import gen_data_collate
from deformation.deformation_net_graph_convolutional_full import DeformationNetworkGraphConvolutionalFull
from deformation.multiview_semantic_discriminator_network import MultiviewSemanticDiscriminatorNetwork
from deformation.semantic_discriminator_net_points import PointsSemanticDiscriminatorNetwork
from deformation.forward_pass import batched_forward_pass, compute_sem_dis_logits
from utils.visualization_tools import save_tensor_img

In [3]:
cfg_path = "configs/test.yaml"

device = torch.device("cuda:0")
cfg = general_utils.load_config(cfg_path, "configs/default.yaml")
batch_size = cfg["semantic_dis_training"]["batch_size"]

In [13]:
class InfiniteDataLoader():
    # a wrapper for a dataloader, which allows batches to be drawn from it infinitely.
    # the order will be random if the dataloder is set to shuffle=True; else it will not be random
    def __init__(self, dataloader):
        self.dataloader = dataloader
        self.dataloader_iter = iter(self.dataloader)
    
    def get_batch(self):
        try:
            data = next(self.dataloader_iter)
        except StopIteration:
            self.dataloader_iter = iter(self.dataloader)
            data = next(self.dataloader_iter)
        return data

# Test Datasets

In [19]:
generation_dataset = datasets.GenerationDataset(cfg)
generation_loader = torch.utils.data.DataLoader(generation_dataset, batch_size=1, num_workers=1, shuffle=True, collate_fn=gen_data_collate, drop_last=True)
infinite_generation_loader = InfiniteDataLoader(generation_loader)

Loading cached generation dataset at caches/generation_dataset_cache_500.pt...


In [21]:
for i in range(600):
    batch = infinite_generation_loader.get_batch()
    print(i)
    print(batch['instance_name'])

0
['193_fake']
1
['307_fake']
2
['206_fake']
3
['296_fake']
4
['083_fake']
5
['001_fake']
6
['197_fake']
7
['452_fake']
8
['314_fake']
9
['033_fake']
10
['248_fake']
11
['230_fake']
12
['385_fake']
13
['330_fake']
14
['293_fake']
15
['268_fake']
16
['294_fake']
17
['006_fake']
18
['397_fake']
19
['378_fake']
20
['423_fake']
21
['472_fake']
22
['005_fake']
23
['262_fake']
24
['424_fake']
25
['255_fake']
26
['266_fake']
27
['160_fake']
28
['460_fake']
29
['123_fake']
30
['431_fake']
31
['096_fake']
32
['217_fake']
33
['187_fake']
34
['000_fake']
35
['320_fake']
36
['147_fake']
37
['451_fake']
38
['073_fake']
39
['275_fake']
40
['259_fake']
41
['076_fake']
42
['003_fake']
43
['498_fake']
44
['122_fake']
45
['178_fake']
46
['449_fake']
47
['163_fake']
48
['372_fake']
49
['402_fake']
50
['135_fake']
51
['379_fake']
52
['105_fake']
53
['370_fake']
54
['092_fake']
55
['222_fake']
56
['448_fake']
57
['063_fake']
58
['467_fake']
59
['207_fake']
60
['022_fake']
61
['100_fake']
62
['030_fake']
63

506
['232_fake']
507
['316_fake']
508
['458_fake']
509
['137_fake']
510
['436_fake']
511
['079_fake']
512
['395_fake']
513
['412_fake']
514
['349_fake']
515
['212_fake']
516
['405_fake']
517
['343_fake']
518
['090_fake']
519
['409_fake']
520
['280_fake']
521
['152_fake']
522
['434_fake']
523
['112_fake']
524
['008_fake']
525
['031_fake']
526
['297_fake']
527
['060_fake']
528
['080_fake']
529
['293_fake']
530
['400_fake']
531
['190_fake']
532
['418_fake']
533
['198_fake']
534
['286_fake']
535
['385_fake']
536
['462_fake']
537
['217_fake']
538
['483_fake']
539
['098_fake']
540
['099_fake']
541
['110_fake']
542
['189_fake']
543
['256_fake']
544
['182_fake']
545
['336_fake']
546
['276_fake']
547
['145_fake']
548
['345_fake']
549
['007_fake']
550
['202_fake']
551
['187_fake']
552
['267_fake']
553
['344_fake']
554
['119_fake']
555
['351_fake']
556
['168_fake']
557
['246_fake']
558
['088_fake']
559
['369_fake']
560
['053_fake']
561
['054_fake']
562
['275_fake']
563
['397_fake']
564
['473_fake

In [5]:
print(len(generation_dataset))
print(generation_dataset[0])
print(generation_dataset[0]["mesh_verts"].shape)

500
{'instance_name': '000_fake', 'mesh': <pytorch3d.structures.meshes.Meshes object at 0x7fce0b038240>, 'mesh_verts': tensor([[5.0947, 4.9838, 5.0988],
        [5.1497, 5.0246, 5.1155],
        [5.0561, 5.0058, 5.0963],
        ...,
        [5.1186, 4.9819, 5.0694],
        [5.0715, 4.9620, 5.1475],
        [5.1361, 4.9888, 5.0745]]), 'image': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
      

In [17]:
real_dataset = datasets.RealDataset(cfg, device)
semantic_dis_loader = torch.utils.data.DataLoader(real_dataset, batch_size=batch_size, num_workers=1, shuffle=True)

Reusing previusly computed points assets...
Loading cached generation dataset at caches/real_dataset/points/data_adversarial_sphere_points_test_real_meshes/cache.pt...


In [7]:
print(real_dataset[0].shape)
print(len(real_dataset))

torch.Size([642, 3])
500


# Test Networks

In [12]:
deform_net = DeformationNetworkGraphConvolutionalFull(cfg, device)
deform_net.to(device)
semantic_dis_net = PointsSemanticDiscriminatorNetwork(cfg)
semantic_dis_net.to(device)

PointsSemanticDiscriminatorNetwork(
  (pointnet_discriminator): Sequential(
    (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): MaxPool1d(kernel_size=642, stride=642, padding=0, dilation=1, ceil_mode=False)
    (7): Flatten()
    (8): Linear(in_features=1024, out_features=512, bias=True)
    (9): LeakyReLU(negative_slope=0.01)
    (10): Linear(in_features=512, out_features=1, bias=True)
    (11): Sigmoid()
  )
)

In [19]:
for real_batch in semantic_dis_loader:
    #real_image_batch = real_batch["mv_images"].to(device)
    #print(real_image_batch.shape)
    print(real_batch.shape)
    real_batch = real_batch.to(device)
    pred_logits_real = semantic_dis_net(real_batch)
    print(pred_logits_real.shape)
    raise

torch.Size([4, 642, 3])
torch.Size([4, 1])


RuntimeError: No active exception to reraise

In [20]:
# TODO: sanity check these images
for gen_batch in generation_loader:
    _, deformed_meshes, _ = batched_forward_pass(cfg, device, deform_net, semantic_dis_net, gen_batch, compute_losses=False)
    pred_logits_fake, semantic_dis_debug_data = compute_sem_dis_logits(deformed_meshes, semantic_dis_net, device, cfg)
    print(pred_logits_fake.shape)
    raise

torch.Size([4, 1])


RuntimeError: No active exception to reraise

# Misc